In [1]:
import __init__
#
from IPython.display import HTML, display
import pandas as pd
import numpy as np
import statsmodels.api as sm
#
from community_analysis import MIN_PICKUP_RATIO
#
def text_display(text, font_size):
    display(HTML('<font size=%d>' % font_size + text + '</font>'))   

In [2]:
def regression(dv, df):
    oc_dv = 'roamingTime' if dv == 'spendingTime' else 'spendingTime'
    rdf = df.copy(deep=True).drop([oc_dv], axis=1)
    candi_dummies = []
    num_iter = 1
    while True:
        for i, vs in enumerate(zip(*rdf.values)):
            if rdf.columns[i] == dv:
                continue
            if sum(vs) > len(rdf) * MIN_PICKUP_RATIO * num_iter:
                candi_dummies.append(rdf.columns[i])
        if len(rdf) <= len(candi_dummies):
            candi_dummies = []
            num_iter += 1
        else:
            break
    y = rdf[dv]
    X = rdf[candi_dummies]
    X = sm.add_constant(X)
    return sm.OLS(y, X, missing='drop').fit()

In [15]:
from community_analysis import tfZ_TP_dpath, tfZ_TP_prefix
#
df = pd.read_csv('%s/%s2009-3.csv' % (tfZ_TP_dpath, tfZ_TP_prefix))

In [16]:
df.head()

,month,day,timeFrame,zi,zj,tfZ,did,spendingTime,roamingTime,32768,...,32442,32469,32502,32509,32529,32537,32542,32647,32656,16375
0,1,1,14,54,12,"(14,54,12)",36942,240,240,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,14,38,45,"(14,38,45)",33720,48,238,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,14,61,27,"(14,61,27)",20677,592,1408,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,14,53,25,"(14,53,25)",29328,596,1428,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,14,41,16,"(14,41,16)",19360,356,987,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
text = '# of drivers in a file: %d' % len(set(df['did']))
text_display(text, 5)

In [18]:
did1 = 19360
did1_df = df[(df['did'] == did1)].copy(deep=True)
did1_df = did1_df.drop(['month', 'day', 'timeFrame', 'zi', 'zj', 'tfZ', 'did'], axis=1)
if '%d' % did1 in did1_df.columns:
    did1_df = did1_df.drop(['%d' % did1], axis=1)

In [19]:
text = '# of observations: %d' % len(did1_df)
text_display(text, 5)
text = '# of indepedent variables:: %d'% len(did1_df.columns)
text_display(text, 5)

In [20]:
text = 'Average: spendingTime %.2f, roamingTime %.2f' % (did1_df['spendingTime'].mean(), did1_df['roamingTime'].mean())
text_display(text, 5)

In [21]:
SP_res = regression('spendingTime', did1_df)
print SP_res.summary()

                            OLS Regression Results                            
Dep. Variable:           spendingTime   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     1.804
Date:                Thu, 19 Jan 2017   Prob (F-statistic):           1.80e-09
Time:                        10:40:18   Log-Likelihood:                -7753.4
No. Observations:                1181   AIC:                         1.594e+04
Df Residuals:                     964   BIC:                         1.704e+04
Df Model:                         216                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        102.5071      9.377     10.932      0.0

In [22]:
RP_res = regression('roamingTime', did1_df)
print RP_res.summary()

                            OLS Regression Results                            
Dep. Variable:            roamingTime   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     1.506
Date:                Thu, 19 Jan 2017   Prob (F-statistic):           2.75e-05
Time:                        10:40:19   Log-Likelihood:                -8566.7
No. Observations:                1181   AIC:                         1.757e+04
Df Residuals:                     964   BIC:                         1.867e+04
Df Model:                         216                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        460.0623     18.669     24.643      0.0

In [23]:
header = ['D.V.', 'SP coef.', 'RP coef.', 'SP pv.', 'RP pv.']
df_data = {k: [] for k in header}
for dv, sp_pv in SP_res.pvalues.iteritems():
    for i, v in enumerate([dv, SP_res.params[dv], RP_res.params[dv], sp_pv, RP_res.pvalues[dv]]):
        if sp_pv < 0.01 and RP_res.pvalues[dv] < 0.01 :
            df_data[header[i]].append(v)
df = pd.DataFrame(df_data)[header]
df

,D.V.,SP coef.,RP coef.,SP pv.,RP pv.
0,const,102.507085,460.062324,2.631940e-26,2.216947e-104
